In [6]:
import pandas as pd
import requests
import gzip
import io
import os
from sqlalchemy import create_engine
import psycopg2

In [17]:
# Fonction qui transforme des fichier .tsv en dataframe puis en table sql 
def Yahya(file_url,rowLimit,engine,nameTable):
        # Téléchargement du fichier
        with requests.get(file_url, stream=True) as r:
            r.raise_for_status()
            with io.BytesIO(r.content) as buf, gzip.open(buf, 'rt', encoding='ISO-8859-1') as gz_file:
                # Transformation en dataframe.
                df = pd.read_csv(gz_file, delimiter='\t', low_memory=False, nrows=rowLimit)

        # Remplacement des  '\\N' par NaN
        df = df.replace('\\N', pd.NA)

        # Transformation du dataframe en table sql 
        df.to_sql(nameTable, engine, if_exists='append', index=False, chunksize=1000)
        print("Data successfully saved to PostgreSQL database.")

# Fonction qui retourne le l'url vers la base de donnée 
def connectEnginePostgresql(username,password,hostname,port,db):
    connection_string = f"postgresql+psycopg2://{username}:{password}@{hostname}:{port}/{db}"
    return connection_string

In [15]:
# Possibilité de mettre une limitation au nombre de ligne pouvant être importer par fichier. 
rowLimit = None

# Url des fichiers
urlList  = [
           'https://datasets.imdbws.com/name.basics.tsv.gz',
           'https://datasets.imdbws.com/title.akas.tsv.gz',
           'https://datasets.imdbws.com/title.basics.tsv.gz',
           'https://datasets.imdbws.com/title.crew.tsv.gz',
           'https://datasets.imdbws.com/title.episode.tsv.gz',
           'https://datasets.imdbws.com/title.principals.tsv.gz',
           'https://datasets.imdbws.com/title.ratings.tsv.gz'
           ]
# Nom des tables 
nameDB   = ['namesBasics','titleAkas','titleBasics','titleCrew','titleEpisode','titlePrincipals','titleRatings']

In [16]:
# L'utilisateur doit inscrire ses données de connexion à sa base de donnée
username = input("nom de l'utilisateur")      
hostname = input('nom du hostname')    
port     = int(input('nom du port'))  
database = input("nom de la base de donnée")
password = input('mot de passe')  

# Connexion à la base de donnée (seulement pour du Postgresql)
connection_string = connectEnginePostgresql(username,password,hostname,port,database)
engine            = create_engine(connection_string)


# boucle for pour chaque URL
try:
    for i,j in zip (urlList,nameDB):
        Yahya(i,rowLimit,engine,j)
except Exception as e:
    print("Error:", e)
finally:
    engine.dispose()
    print("Connection closed")
        

Error: 'utf-8' codec can't decode byte 0xe9 in position 84: invalid continuation byte
Connection closed
